In [1]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 MB 17.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-4.0.0-py2.py3-none-any.whl size=434741300 sha256=ecc728438399fae24e927966887b45685634c91482693e856a75d5c8f7f3d550
  Stored in directory: /Users/polgarcia/Library/Caches/pip/wheels/2d/77/9b/12660be70f7f447940a0caede37ae208b2e0d1c8487dce52a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("landingtoformatted").getOrCreate()

#change the distribucio o  with the accent.
landing_zone = 'landing_zone'
formatted_zone = 'formatted_zone'

datasets = [
    "landing_zone/income/2007_Distribucio_territorial_renda_familiar.csv",
    "landing_zone/income/2008_Distribucio_territorial_renda_familiar.csv",
    "landing_zone/income/2009_Distribucio_territorial_renda_familiar.csv",
    "landing_zone/income/2010_Distribucio_territorial_renda_familiar.csv",
    "landing_zone/income/2011_Distribucio_territorial_renda_familiar.csv",
    "landing_zone/income/2012_Distribucio_territorial_renda_familiar.csv",
    "landing_zone/income/2013_Distribucio_territorial_renda_familiar.csv",
    "landing_zone/income/2014_Distribucio_territorial_renda_familiar.csv",
    "landing_zone/income/2015_Distribucio_territorial_renda_familiar.csv",
    "landing_zone/income/2016_Distribucio_territorial_renda_familiar.csv",
    "landing_zone/income/2017_Distribucio_territorial_renda_familiar.csv"
]
initial_df = spark.read.csv(datasets[0], header=True, inferSchema=True)
for dataset in datasets[1:]:
    df_next = spark.read.csv(dataset, header=True, inferSchema=True)
    initial_df = initial_df.union(df_next)

In [6]:
initial_df.show(100)

+----+--------------+-------------------+----------+--------------------+--------+-------------------------+
| Any|Codi_Districte|      Nom_Districte|Codi_Barri|           Nom_Barri|Població|Índex RFD Barcelona = 100|
+----+--------------+-------------------+----------+--------------------+--------+-------------------------+
|2007|             1|       Ciutat Vella|         1|            el Raval|   46595|                     64.7|
|2007|             1|       Ciutat Vella|         2|      el Barri Gòtic|   27946|                     86.5|
|2007|             1|       Ciutat Vella|         3|      la Barceloneta|   15921|                     66.7|
|2007|             1|       Ciutat Vella|         4|Sant Pere, Santa ...|   22572|                     80.2|
|2007|             2|           Eixample|         5|       el Fort Pienc|   31521|                    107.9|
|2007|             2|           Eixample|         6|  la Sagrada Família|   52185|                    101.8|
|2007|             